In [ ]:
# Install packages from requirements.txt
%pip install -r requirements.txt

In [1]:
# Set the working directory and adjust the Python path
import os
import sys

# Set the working directory to the src directory
os.chdir('./')
sys.path.append(os.getcwd())

from scripts.utils.generators import *
from scripts.test.test_graph_generators import *
import pprint


In [2]:
map_graph = generate_square_city_graph()
pprint.pprint(map_graph)

{'connections': [(0, 10),
                 (0, 1),
                 (1, 11),
                 (1, 0),
                 (1, 2),
                 (2, 12),
                 (2, 1),
                 (2, 3),
                 (3, 13),
                 (3, 2),
                 (3, 4),
                 (4, 14),
                 (4, 3),
                 (4, 5),
                 (5, 15),
                 (5, 4),
                 (5, 6),
                 (6, 16),
                 (6, 5),
                 (6, 7),
                 (7, 17),
                 (7, 6),
                 (7, 8),
                 (8, 18),
                 (8, 7),
                 (8, 9),
                 (9, 19),
                 (9, 8),
                 (10, 0),
                 (10, 20),
                 (10, 11),
                 (11, 1),
                 (11, 21),
                 (11, 10),
                 (11, 12),
                 (12, 2),
                 (12, 22),
                 (12, 11),
                 (12, 1

In [3]:
buses_graph = generate_bus_line_square_city()
pprint.pprint(buses_graph)

[{'connections': [(5, 1005),
                  (1005, 1015),
                  (15, 1015),
                  (1015, 1025),
                  (25, 1025),
                  (1025, 1035),
                  (35, 1035),
                  (1035, 1045),
                  (45, 1045),
                  (1045, 1055),
                  (55, 1055),
                  (1055, 1065),
                  (65, 1065),
                  (1065, 1075),
                  (75, 1075),
                  (1075, 1085),
                  (85, 1085),
                  (1085, 1095)],
  'name': 'bus line UNIQUE',
  'node_bus_index': array([1005, 1015, 1025, 1035, 1045, 1055, 1065, 1075, 1085, 1095]),
  'node_map_index': array([ 5, 15, 25, 35, 45, 55, 65, 75, 85, 95]),
  'weights': [0.13500000000000004,
              0.05,
              0.13500000000000004,
              0.05,
              0.13500000000000004,
              0.05,
              0.13500000000000004,
              0.05,
              0.13500000000000004,


In [4]:
merge_bus_and_map_graph(map_graph,buses_graph)
pprint.pprint(map_graph)

{'buses': [{'connections': [(5, 1005),
                            (1005, 1015),
                            (15, 1015),
                            (1015, 1025),
                            (25, 1025),
                            (1025, 1035),
                            (35, 1035),
                            (1035, 1045),
                            (45, 1045),
                            (1045, 1055),
                            (55, 1055),
                            (1055, 1065),
                            (65, 1065),
                            (1065, 1075),
                            (75, 1075),
                            (1075, 1085),
                            (85, 1085),
                            (1085, 1095)],
            'name': 'bus line UNIQUE',
            'node_bus_index': array([1005, 1015, 1025, 1035, 1045, 1055, 1065, 1075, 1085, 1095]),
            'node_map_index': array([ 5, 15, 25, 35, 45, 55, 65, 75, 85, 95]),
            'weights': [0.13500000000000004,
